In [6]:
import pandas as pd
from unicodedata2 import normalize

In [7]:
def normalizar_colunas(dataframe):
    lista_colunas = []
    for coluna in dataframe.columns:
        nova_coluna = normalize('NFKD', coluna).encode('ascii', 'ignore').decode('utf-8')#.replace(' ', '_')
        lista_colunas.append(nova_coluna)
    dataframe.columns = lista_colunas
    return dataframe

def salvamento_csv(dataframe, file_name):
    # TODO Verificação da existência da pasta (ou criação, caso não exista)
    dataframe.to_csv(f"..\\CSV\\01.bronze\\bronze_{file_name}.csv", index=False)

In [10]:
# TODO Carregamento dinâmico dos arquivos na pasta "Excel"
arquivo_completo = pd.read_excel(r"..\Excel\Sumula_Teste_T01B01.xlsx", sheet_name=None, engine="calamine")

df_individual = arquivo_completo["Individual"]
normalizar_colunas(df_individual)

df_times = arquivo_completo["Times"]
normalizar_colunas(df_times)

df_info = arquivo_completo["Informações"]
normalizar_colunas(df_info)


salvamento_csv(df_individual, "desempenho_individual")
salvamento_csv(df_times, "desempenho_times")
salvamento_csv(df_info, "info_partidas")


In [ ]:
df_individual

In [ ]:
df_times

In [ ]:
df_info